## QHyper usecase

### Create instance of problem

In [1]:
from QHyper.problems import KnapsackProblem, TSPProblem

PROBLEM_TYPE = 'knapsack' # 'tsp'


# Each problem needs different parameters, because it depends on the number
# of variables and constraints
# Number of hyper_optimizer_bounds depends on the number of constraints,
# because each constraint requires one weights, and objective function also
# requires one

if PROBLEM_TYPE == 'knapsack':
    # Create knapsack with 3 items: 2 with weights 1 and
    # value 2 - (1, 2), and one with weight 1 and value 1 - (1, 1)
    problem = KnapsackProblem(max_weight=2, items=[(1, 2), (1, 2),(1, 1)])

    problem_config = {  # This is going to be used later
        'type': 'knapsack',
        'max_weight': 2,
        'items': [(1, 2), (1, 2),(1, 1)]
    }

    params_config = {
        'angles': [[0.5]*5, [1]*5],
        'hyper_args': [1, 2.5, 2.5],
    }
    hyper_optimizer_bounds = 3*[(1, 10)]
    penalty = 2

elif PROBLEM_TYPE == 'tsp':
    # Create Traveling Salesmam Problem with 3 cities
    problem = TSPProblem(number_of_cities=3)

    problem_config = {  # This is going to be used later
        'type': 'tsp',
        'number_of_cities': 3,
    }

    params_config = {
        'angles': [[0.5]*5, [1]*5],
        'hyper_args': [1, 2, 2, 2, 2],
    }
    hyper_optimizer_bounds = 5*[(1, 10)]
    penalty = -1


In [2]:
print(f"Variables used to describe objective function"
      f"and constraints: {problem.variables}")
print(f"Objective function: {problem.objective_function}")
print("Constraints:")
for constraint in problem.constraints:
    print(f"    {constraint}")


Variables used to describe objective functionand constraints: (x0, x1, x2, x3, x4)
Objective function: Polynomial(terms=defaultdict(<class 'float'>, {('x0',): -2.0, ('x1',): -2.0, ('x2',): -1.0}))
Constraints:
    Polynomial(terms=defaultdict(<class 'float'>, {('x3',): -1.0, ('x4',): -1.0, (): 1.0})) == Polynomial(terms=defaultdict(<class 'float'>, {}))
    Polynomial(terms=defaultdict(<class 'float'>, {('x0',): -1.0, ('x1',): -1.0, ('x2',): -1.0, ('x3',): 1.0, ('x4',): 2.0})) == Polynomial(terms=defaultdict(<class 'float'>, {}))


### Use VQA to solve knapsack problem

In [3]:
# Create a VQA instance with HQAOA as PQC and scipy optimizer
# This can be done in two various way
# 1. Providing dict with config (usefull to save experiment confing in e.g JSON)
from QHyper.solvers import solver_from_config

# While we could use problem instace to create solver, the better approach
# is to use config, because it is easier to save it in JSON file and load it later

solver_config = {
    "solver": {
        "type": "vqa",
        "optimizer": {
            "type": "scipy",
            "maxfun": 200,
        },
        "pqc": {
            "type": "wfqaoa",
            "layers": 5,
            "limit_results": 20,
            "penalty": penalty,
            "backend": "default.qubit",
        },
        "params_inits": params_config
    },
    "problem": problem_config
}

vqa = solver_from_config(solver_config)

# # 2. Providing actual isntance of each class like VQA and Optimizer
# NOT RECOMMENDED

# from QHyper.solvers.vqa import VQA
# from QHyper.solvers.vqa.pqc import HQAOA
# from QHyper.optimizers import ScipyOptimizer

# vqa = VQA(problem, HQAOA(layers=5, penalty=penalty, backend='default.qubit'),
#           ScipyOptimizer(maxfun=200))


In [4]:
# Run VQA with provided initial parameters
solver_results = vqa.solve()

print("Solver results:")
print(f"Probabilities: {solver_results.results_probabilities}")
print(f"Best params: {solver_results.params}")


AttributeError: 'tensor' object has no attribute 'items'

In [ ]:
from QHyper.util import (
    weighted_avg_evaluation, sort_solver_results, add_evaluation_to_results)

# Evaluate results with weighted average evaluation
print("Evaluation:")
print(weighted_avg_evaluation(
    solver_results.results_probabilities, problem.get_score,
    penalty=0, limit_results=10, normalize=True
))
print("Sort results:")
sorted_results = sort_solver_results(
    solver_results.results_probabilities, limit_results=10)

# Add evaluation to results
results_with_evaluation = add_evaluation_to_results(
    sorted_results, problem.get_score, penalty=penalty)

for result, (probability, evaluation) in results_with_evaluation.items():
    print(f"Result: {result}, "
          f"Prob: {probability:.5}, "
          f"Evaluation: {evaluation}")


Evaluation:
-1.675280267157278
Sort results:
Result: 11101, Prob: 0.07454, Evaluation: 2
Result: 01101, Prob: 0.070341, Evaluation: -3
Result: 10101, Prob: 0.070341, Evaluation: -3
Result: 11001, Prob: 0.066772, Evaluation: -4
Result: 11010, Prob: 0.048268, Evaluation: 2
Result: 00110, Prob: 0.046071, Evaluation: -1
Result: 00101, Prob: 0.040596, Evaluation: 2
Result: 01010, Prob: 0.038698, Evaluation: -2
Result: 10010, Prob: 0.038698, Evaluation: -2
Result: 10011, Prob: 0.036928, Evaluation: 2


#### Using hyper optimizers

In [ ]:
# Additionally other optimizer can be used to tune some parameters, in below
# example, Random optimzier will change weights (hyper_args) and choose ones
# that gives the best results after runnign 200 iteration of scipy minimizer

solver_config = {
    "solver": {
        "type": "vqa",
        "optimizer": {
            "type": "scipy",
            "maxfun": 200,
        },
        "pqc": {
            "type": "wfqaoa",
            "layers": 5,
            "limit_results": 20,
            "penalty": penalty,
        },
        "params_inits": params_config,
        "hyper_optimizer": {
            "type": "random",
            "processes": 5,
            "number_of_samples": 100,
            "bounds": hyper_optimizer_bounds,
        }
    },
    "problem": problem_config
}
vqa = solver_from_config(solver_config)


In [ ]:
solver_results = vqa.solve()

print("Solver results:")
print(f"Probabilities: {solver_results.results_probabilities}")
print(f"Best params: {solver_results.params}")


{'angles': [[0.5, 0.5, 0.5, 0.5, 0.5], [1, 1, 1, 1, 1]], 'hyper_args': [1, 2.5, 2.5]}


100%|██████████| 100/100 [02:23<00:00,  1.44s/it]


Solver results:
Probabilities: {'00000': 0.030243821901753693, '00001': 0.05351321410987908, '00010': 0.02394599771619717, '00011': 0.028966852103382276, '00100': 0.023983437518947333, '00101': 0.011982768728699765, '00110': 0.050188054411493696, '00111': 0.00981095350279921, '01000': 0.008080309672715637, '01001': 0.0031252226877130584, '01010': 0.0052025284895525865, '01011': 0.008355255310373825, '01100': 0.009191255796983188, '01101': 0.18183158517032383, '01110': 0.008372051927368987, '01111': 0.008006135446706137, '10000': 0.008080309672715662, '10001': 0.0031252226877130597, '10010': 0.005202528489552583, '10011': 0.008355255310373845, '10100': 0.00919125579698322, '10101': 0.18183158517032388, '10110': 0.008372051927368986, '10111': 0.008006135446706135, '11000': 0.012850789832666397, '11001': 0.2080008475543236, '11010': 0.008567833389710549, '11011': 0.02018794745020287, '11100': 0.008510368946718203, '11101': 0.01750844960096943, '11110': 0.026359289919377422, '11111': 0.001

In [ ]:
print("Evaluation:")
print(weighted_avg_evaluation(
    solver_results.results_probabilities, problem.get_score,
    penalty=0, limit_results=10, normalize=True
))
print("Sort results:")
sorted_results = sort_solver_results(
    solver_results.results_probabilities, limit_results=10)

results_with_evaluation = add_evaluation_to_results(
    sorted_results, problem.get_score, penalty=penalty)

for result, (probability, evaluation) in results_with_evaluation.items():
    print(f"Result: {result}, "
          f"Prob: {probability:.5}, "
          f"Evaluation: {evaluation}")


Evaluation:
-2.4394450528466396
Sort results:
Result: 11001, Prob: 0.208, Evaluation: -4
Result: 10101, Prob: 0.18183, Evaluation: -3
Result: 01101, Prob: 0.18183, Evaluation: -3
Result: 00001, Prob: 0.053513, Evaluation: 2
Result: 00110, Prob: 0.050188, Evaluation: -1
Result: 00000, Prob: 0.030244, Evaluation: 0
Result: 00011, Prob: 0.028967, Evaluation: 2
Result: 11110, Prob: 0.026359, Evaluation: 2
Result: 00100, Prob: 0.023983, Evaluation: 2
Result: 00010, Prob: 0.023946, Evaluation: 2
